In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.models import LdaModel

In [ ]:
import cx_Oracle
import pandas as pd
conn = cx_Oracle.connect("admin", "Onlin3#onlin3#", "adbnews_high")

query = """
    SELECT *
    FROM tb_news_valor_economico
    --WHERE rownum <=10
"""
#curr = conn.cursor()
#curr.execute(query)

news_data = pd.read_sql(query, con=conn)
conn.close()